In [1]:
import tkinter as tk
from tkinter import Canvas, Tk, mainloop, Toplevel
from tkinter import filedialog
from tkinter import messagebox
from PIL import Image, ImageTk, ImageDraw
import time
from tkinter.font import Font

In [2]:
def Crop_img():
    
    root = Toplevel()
    root.title('Crop Image')
    crop_img = Image.open(imgpath_Var.get())
    width, height = crop_img.size

    ca = Canvas(root, width=width, height=height)
    ca.pack()
    photoimg = ImageTk.PhotoImage("RGB", crop_img.size)
    photoimg.paste(crop_img)
    ca.create_image(width//2,height//2, image=photoimg)

    loc = []
    images = []
    temp_var = tk.IntVar()
    temp_var.set(0)

    def create_rectangle(x1, y1, x2, y2, **kwargs):
        if 'alpha' in kwargs:
            alpha = int(kwargs.pop('alpha') * 255)
            fill = kwargs.pop('fill')
            fill = root.winfo_rgb(fill) + (alpha,)
            image = Image.new('RGBA', (x2-x1, y2-y1), fill)
            images.append(ImageTk.PhotoImage(image))
            ca.create_image(x1, y1, image=images[-1], anchor='nw')
        ca.create_rectangle(x1, y1, x2, y2, **kwargs)

    def callback1(event):
        
        if len(loc)>=1:
            loc.clear()
        
        print('左鍵當前位置為：',event.x, event.y)
        x1,y1 = (event.x-2),(event.y-2)
        x2,y2 = (event.x+2),(event.y+2)
        create_rectangle(x1,y1,x2,y2,fill="red")
        loc.append((event.x, event.y))

    def callback2(event):
        print('右鍵當前位置為：',event.x, event.y)
        x1,y1 = (event.x-2),(event.y-2)
        x2,y2 = (event.x+2),(event.y+2)
        create_rectangle(x1,y1,x2,y2,fill="red")
        create_rectangle(loc[0][0],loc[0][1], event.x, event.y, fill="yellow", alpha=.5)

        img_copy = crop_img.copy()
        img_crop = img_copy.crop((loc[0][0], loc[0][1], event.x, event.y))
        time_str = time.strftime("%Y%m%d_%H%M%S", time.localtime()) #現在時間格式化成20160320 114539形式
        img_crop.save("Teeth"+ time_str +".jpg")
        
        img_crop_resize = img_crop.resize((40,80))
        imgcroptk = ImageTk.PhotoImage(img_crop_resize)
        
        temp_number = temp_var.get()
        
        label_img_list[temp_number].config(image = imgcroptk)
        label_img_list[temp_number].image = imgcroptk
        
        temp_var.set(temp_number+1)
        
        loc.clear()

    ca.bind('<Button-1>',callback1)
    ca.bind('<Button-3>',callback2)
    ca.pack()
    mainloop()

In [3]:
#建立選擇欄
def NewFile():
    print("New File!")
def About():
    messagebox.showinfo(title='About', message="本研究東華大學測試使用!")

In [4]:
def Open_img():
    
    filename = filedialog.askopenfilename(title='Select file', filetypes=[("all files","*.*"),("bmp files","*.bmp"),("png files","*.png"),("jpeg files","*.jpg")])
    
    img = Image.open(filename).resize((300,200))
    imgtk = ImageTk.PhotoImage(img)
    label_img.config(image = imgtk)
    label_img.image = imgtk
    imgpath_Var.set(filename)

In [5]:
app = tk.Tk()
app.title('Teeth AI')
app.geometry('500x480')

#建立菜單
menu = tk.Menu(app)
app.config(menu=menu)

#建立菜單的file
file_menu = tk.Menu(menu, tearoff = 0) #是否有虛線，無虛線則無法分離視窗
menu.add_cascade(label="File", menu=file_menu)

#file建立新增
file_menu.add_command(label="New", command=NewFile)

#file建立虛線
file_menu.add_separator()

#file建立離開
file_menu.add_command(label="Exit", command=app.destroy)

#建立菜單的幫助
help_menu = tk.Menu(menu, tearoff = 0)
menu.add_cascade(label="Help", menu=help_menu)
help_menu.add_command(label="About", command=About)

#設立字體
subtitle_label_Font_1 = Font(family="Times", size=12)

#建立畫布與位置
frame_title = tk.Frame(app)
frame_title.grid(column=0, row=0, ipadx=5, pady=5)

frame_img = tk.LabelFrame(app, text='Image show')  
frame_img.grid(column=0, row=1, ipadx=0, padx=10, pady=10, sticky=tk.W+tk.N)  

frame_but = tk.LabelFrame(app, text='Open/Crop') 
frame_but.grid(column=1, row=1, ipadx=0, padx=10, pady=10, sticky=tk.W+tk.N)  

frame_output = tk.LabelFrame(app, text='Crop Image')
frame_output.grid(column=0, row=2, ipadx=0, padx=10, pady=10, sticky=tk.W+tk.N)

# 建立文字
label_Subtitle = tk.Label(frame_img, text="Please enter the image.", font=subtitle_label_Font_1)
label_Subtitle.grid(column=0, row=0, ipadx=0, pady=0)

#GUI影像
img_path = "p-img.png"
img = Image.open(img_path).resize((300,200))
imgtk = ImageTk.PhotoImage(img)
label_img = tk.Label(frame_img, image=imgtk)
label_img.grid(column=0, row=1, ipadx=10, pady=10)

# 影像放置位置
img_path_list = ["p-img.png","p-img.png","p-img.png","p-img.png","p-img.png","p-img.png"]
imgtk_list, label_img_list = [], []

for i in range(len(img_path_list)):
    simg = Image.open(img_path_list[i]).resize((40,80))
    simgtk = ImageTk.PhotoImage(simg)
    imgtk_list.append(simgtk)

for i in range(len(img_path_list)):
    slabel_img = tk.Label(frame_output, image=imgtk_list[i])
    label_img_list.append(slabel_img)
    
for i in range(len(img_path_list)):
    label_img_list[i].grid(column=i, row=0, ipadx=5, pady=5)

#影像位置變數
imgpath_Var = tk.StringVar()

#插入影像按鈕
imgButton = tk.Button(frame_but, text = 'Open Image',command = Open_img)
imgButton.grid(column=0, row=1, padx=10, pady=10)

#裁剪按鈕
cropButton = tk.Button(frame_but, text = 'Crop Image',command = Crop_img)
cropButton.grid(column=0, row=2, padx=10, pady=10)

mainloop()

左鍵當前位置為： 146 79
右鍵當前位置為： 430 529
左鍵當前位置為： 464 126
右鍵當前位置為： 737 567
左鍵當前位置為： 137 77
右鍵當前位置為： 460 550
左鍵當前位置為： 469 122
右鍵當前位置為： 756 583
